## Adjoint derivatives tutorial: A shallow water model

In this tutorial we aim to demonstrate to users how one can use Enzyme + Checkpointing in Julia to compute derivatives of a shallow water model and provide physical interpretations of the computed derivatives. 

### Model overview

The differentiable model we'll examine in this tutorial is a **shallow water model**. This just means we're examining a simplified ocean model: imagine a box of fluid with wind-driven circulation. The equations of motion are given by 
\begin{align}
        \frac{\partial}{\partial t} u + u \frac{\partial}{\partial x} u + v \frac{\partial}{\partial y} u - fv &= - g \frac{\partial}{\partial x} \eta + F_x + M_x \\
        \frac{\partial}{\partial t} v + u \frac{\partial}{\partial x} v + v \frac{\partial}{\partial y} v + fv &= - g \frac{\partial}{\partial y} \eta + F_y + M_y \\
        \frac{\partial }{\partial t} \eta + \nabla \cdot (h \mathbf{u}) &= 0
\end{align}
with the model parameters defined as 
\begin{align*}
 &\text{Variable} &&\text{Definition} \\ 
 \hline
 &f = f(y) = f_0 + \beta y &&\text{Coriolis function} \\
 &f_0 &&\text{Coriolis parameter} \\
 &\beta &&\text{Rossby parameter} \\
 &g &&\text{Gravity constant} \\
 &h(t,x,y) = \eta(t, x, y) + H &&\text{Surface height} \\ 
 &\mathbf{F} = (F_x, F_y) &&\text{Wind forcing} \\
 &\mathbf{M} = (M_x, M_y) &&\text{Momentum mixing and bottom friction} 
 \end{align*}

#### References:

[Klöwer, M., Hatfield, S., Croci, M., Düben, P. D., & Palmer, T. N. (2022). Fluid simulations accelerated with 16 bits: Approaching 4x speedup on A64FX by squeezing ShallowWaters. jl into Float16. Journal of Advances in Modeling Earth Systems, 14(2), e2021MS002684.](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2021MS002684)

### Adding necessary Julia packages

In order to run the tutorial one needs the following packages:

1. ShallowWaters.jl -- this is the package that contains the model code.
2. Enzyme.jl -- Enzyme will do the differentiating for us.
3. Checkpointing.jl -- because of the nature of adjoint methods, we need to "checkpoint" the adjoint problem (i.e. break the backwards run into multiple shorter backwards runs)
4. Plots.jl -- for making pretty pictures at the end.

It's also recommended that you're on Julia v1.10.0, earlier versions might not work

In [ ]:
using ShallowWaters
using Enzyme
using Checkpointing
using Plots

include("ShallowWaters.jl")